# Signal processing course 2018/2019-1 @ ELTE
# Assignment 2
## 10.01.2018

In [ ]:
import numpy as np

import seaborn as sns
import matplotlib.cm as cm
import matplotlib.pyplot as plt

In [ ]:
# Initialize seaborn with custom settings
# Facecolor values from S. Conradi @S_Conradi/@profConradi
custom_settings = {
    'figure.facecolor': '#f4f0e8',
    'axes.facecolor': '#f4f0e8',
    'axes.edgecolor': '0.7',
    'axes.linewidth' : '2',
    'grid.color': '0.7',
    'grid.linestyle': '--',
    'grid.alpha': 0.6,
}
sns.set_theme(palette=sns.color_palette('deep', as_cmap=False),
              rc=custom_settings)

## Task 2 $-$ Introduction to Fourier series
### Definition of Fourier series

Let us consider a real-valued function $f(x)$ of the variable $x$, where $x$ is a real number. The function $f$ is assumed to be integrable over the interval $[x_{0}, x_{0} + T]$, with $x_{0}$​ and $T$ being specific real numbers. The primary objective of Fourier series is to express the function $f$ over this interval as an infinite series of harmonically related sinusoidal functions. These sinusoidal functions are fundamentally sines and cosines, which more properly, can be expressed using complex exponentials.

The concept of periodicity is central to the Fourier series. Specifically, outside the given interval $[x_{0}, x_{0} + T]$, the Fourier series representation of $f$ exhibits periodicity with a period of $T$. This periodic nature is characterized by a frequency of $1/T$​. Consequently, if the function $f$ itself is periodic with the same period $T$, then the Fourier series approximation is not only applicable within the interval $[x_{0}, x_{0} + T]$ but extends its validity across the entire real line.

#### Partial sum and convergence
To construct the Fourier series, one often starts with a finite summation, known as a **partial sum**, expressed in its **amplitude-phase form** as

$$
    f_{N}(x)
    =
    \frac{A_{0}}{2} + \sum_{k = 1}^{N} A_{k} \cdot \sin \left( \frac {2\pi kx}{T} + \phi _{k} \right)
    \quad
    \text{for integers}\ N\ \geq\ 1.
$$

Here, $A_{k}$​ and $\phi_{k}$​ represent the amplitude and phase of the $k$-th harmonic component, respectively. The "partial sum" involves summing a finite number of the harmonically related sinusoids (or complex exponentials) to approximate the function $f$. The partial sum, denoted as $f_{N} (x)$, represents the Fourier series approximation of $f(x)$ using the first $N$ terms. As $N$ increases, the approximation $f_{N} ​(x)$ converges to the actual function $f(x)$ over the interval $[x_{0}, x_{0} + T]$.

#### Standard sine-cosine form
By expressing the higher-order terms (for $N \geq 1$) as a combination of sine and cosine terms and substituting $\omega_{0} = 2\pi / T$, we arrive at the sine-cosine form of the partial sum

$$
    f_{N}(x)
    =
    \overbrace{a_{0}}^{A_{0}}/2 + \sum_{k = 1}^N
    \left(
        \overbrace{a_{k}}^{A_{k} \sin(\phi_{k})} \cdot \cos \left( \omega_{0} k x \right)
        +
        \overbrace{b_{k}}^{A_{k} \cos(\phi_{k})} \cdot \sin \left( \omega_{0} k x \right)
    \right).
$$

Here, we used several substitutions, indicated in the equation above. The inverse relationships between the old and new coefficients introduced here are

$$
    A_{k} = \sqrt{a_{k}^{2} + b_{k}^{2}}
    \qquad
    \phi _{k} = \operatorname{arctan} \left( \frac{a_{k}}{b_{k}} \right)
$$

In [ ]:
def fourier_sin(t, a, b, N, T, fs=128):
    '''
    Approximates a target periodic :math:`f_N(t)` function using the
    sine-cosine form of the Fourier series in the interval
    :math:`[t_0, t_0+T]`.

    Parameters
    ----------
    t : float or array-like
        Sampling times corresponding to the evaluated signal
    a : array-like
        List of :math:`a_k` coefficients
    b : array-like
        List of :math:`b_k` coefficients
    N : int
        The highest order of harmonics to be considered in the
        approximation
    T : float
        Period of the evaluated function
    fs : float
        Sampling frequency of the evaluated function
    '''
    # Do not enable larger values for N than the number of values in the
    # input `a` or `b` arrays. Also prevent N < 1 values.
    assert (N >= 1) | (len(a) >= 1), 'N should be larger than or equal to 1!'
    N = np.min((N, len(a)))

    # Handle both scalar and array inputs
    FCHECK = np.isscalar(t)
    if FCHECK:
        t = np.array([t])

    # Employ the substitution P(t) = w0 * t
    P = 2*np.pi / T * t
    # Calculate the approximation up to an order N
    fN = np.zeros_like(t, dtype=float) + a[0]/2
    for k in range(1, N):
        fN += a[k] * np.cos(P * k) + b[k] * np.sin(P * k)

    if FCHECK:
        return fN[0]
    return fN

#### Exponential form
This can be further improved by expanding the sinusodial functions to complex exponentials using Euler's formula, resulting in the exponential form

$$
    f_{N}(x)
    =
    \sum_{k = -N}^{N} c_{k} \cdot e^{i \omega_{0} k x},
$$

where

$$
    c_k \ \stackrel{\mathrm{def}}{=} \
    \begin{cases}
        \frac{A_{k}}{2i} e^{i \phi_{k}} = \frac{1}{2} (a_{k} - i b_{k}) & k > 0 \\
        \frac{1}{2} A_0 = \frac{1}{2} a_{0} & k = 0 \\
        c_{|k|}^* & k < 0
    \end{cases}
$$

In [ ]:
def c_k(k, a, b):
    '''
    Calculates the conditional value of :math:`c_k` from its definition.

    Parameters
    ----------
    k : int
        The respective value of the :math:`k` running index
    a : array-like
        List of :math:`a_k` coefficients
    b : array-like
        List of :math:`b_k` coefficients
    '''
    if k > 0:
        return 0.5 * (a[k] - 1j * b[k])
    elif k == 0:
        return 0.5 * a[0]
    elif k < 0:
        return 0.5 * (a[k] + 1j * b[k])

In [ ]:
def fourier_exp(t, a, b, N, T, fs=128):
    '''
    Approximates a target periodic :math:`f_N(t)` function using the
    exponential form of the Fourier series in the interval
    :math:`[t_0, t_0+T]`.

    Parameters
    ----------
    t : float or array-like
        Sampling times corresponding to the evaluated signal
    a : array-like
        List of :math:`a_k` coefficients
    b : array-like
        List of :math:`b_k` coefficients
    N : int
        The highest order of harmonics to be considered in the
        approximation
    T : float
        Period of the evaluated function
    fs : float
        Sampling frequency of the evaluated function
    '''
    # Do not enable larger values for N than the number of values in the
    # input `a` or `b` arrays. Also prevent N < 1 values.
    assert (N >= 1) | (len(a) >= 1), 'N should be larger than or equal to 1!'
    N = np.min((N, len(a)))

    # Handle both scalar and array inputs
    FCHECK = np.isscalar(t)
    if FCHECK:
        t = np.array([t])

    # Employ the substitution P(t) = i * w0 * t
    P = 1j * 2*np.pi / T * t
    # Sum over 2N+1 number of `k` indices, running between ]-N -> N[
    fN = np.zeros_like(t, dtype=np.complex128)
    for k in range(-N+1, N):
        fN += c_k(k, a, b) * np.exp(P * k)

    if FCHECK:
        return fN[0]
    return fN

#### Convergence and Fourier coefficients
As $N$ approaches infinity, the partial sum $f_{N}​(x)$ converges to the full Fourier series, providing a complete representation of $f(x)$ over the interval $[x_{0}, x_{0} + T]$ and, under periodicity conditions, over the entire real line. In this limit, using the sine-cosine or the exponential form, the coefficients (known as Fourier coefficients) can be computed as follows:

$$
\begin{alignat*}{3}
    a_{k}
    &=
    \frac{2}{T} \int_{x_{0}}^{x_{0} + T} f(x) \cdot \cos \left( \frac{2\pi k x}{T} \right) \operatorname{dx}
    &&\equiv
    \frac{2}{T} \int_{x_{0}}^{x_{0} + T} f(x) \cdot \cos \left( k \omega_{0} x \right) \operatorname{dx} \\
    b_{k}
    &=
    \frac{2}{T} \int_{x_{0}}^{x_{0} + T} f(x) \cdot \sin \left( \frac{2\pi k x}{T} \right) \operatorname{dx}
    &&\equiv
    \frac{2}{T} \int_{x_{0}}^{x_{0} + T} f(x) \cdot \sin \left( k \omega_{0} x \right) \operatorname{dx} \\
    c_{k}
    &=
    \frac{1}{T} \int_{x_{0}}^{x_{0} + T} f(x) \cdot e^{-i \frac{2\pi k x}{T}} \operatorname{dx}
    &&\equiv
    \frac{1}{T} \int_{x_{0}}^{x_{0} + T} f(x) \cdot e^{-i k \omega_{0} x} \operatorname{dx}
\end{alignat*}
$$

#### Calculating Fourier coefficients numerically<sup>[\[1\]](#footnote1)</sup>

Fourier coefficients can be approximated by numeric integration. Using the trapezoidal rule in the case of the $N + 1$ number of equidistant grid points, defined as

$$
    t_{\nu} = \nu h
$$

where $\nu = 0, 1, \dots, N$ and $h = 2 \pi / N$, the following approximate formulae can be obtained:

$$
\begin{align}
    a_{k} &\approx \tilde{a}_{k} = \frac{2}{N} \sum_{\nu = 1}^{N} f(t_{\nu}) \cos \left( k t_{\nu} \right), \\
    b_{k} &\approx \tilde{b}_{k} = \frac{2}{N} \sum_{\nu = 1}^{N} f(t_{\nu}) \sin \left( k t_{\nu} \right),
\end{align}
$$

where $k = 0, 1, \dots, N$. By analogy, the following formula can be used for the $c_{k}$ coefficients:

$$
    c_{k} \approx \tilde{c}_{k}
    =
    \frac{1}{N} \sum_{\nu = 0}^{N - 1} f(t_{\nu}) e^{-i k t_{\nu}}
$$

where again, $k = 0, 1, \dots, N$ and $\nu = 0, 1, \dots, N-1$.

<a name="footnote1">1</a>: Bronstein, 2009. Typotex, pp. 950-952

In [ ]:
from scipy.integrate import quad

In [ ]:
def fourier_coeffs(f, T, N, t0=0, *args):
    '''
    Calculates the Fourier coefficients :math:`a_k`, :math:`b_k` up to
    an arbitrary order :math:`N` for a given periodic signal using
    numerical integration.

    Parameters
    ----------
    f : callable
        The target periodic function
    T : float
        Period of the function
    N : int
        The highest order of harmonics to be considered in the
        approximation
    t0 : float
        The starting point of the period (default is 0)
    args : tuple, optional
        Extra arguments to pass to `f` inside the lambda func. Should
        not be confused with the optional `args` argument of
        `scipy.integrate.quad()` inside the decorator functions.
    '''
    # Employ the substitution defined above
    w0 = 2*np.pi / T

    def a_k(k):
        ''' Function to integrate for a_k '''
        func = lambda t: f(t, *args) * np.cos(k * t * w0)
        return quad(func, t0, t0 + T)[0] * 2 / T

    def b_k(k):
        ''' Function to integrate for b_k '''
        func = lambda t: f(t, *args) * np.sin(k * t * w0)
        return quad(func, t0, t0 + T)[0] * 2 / T

    a = np.array([a_k(k) for k in range(N)])
    b = np.array([b_k(k) for k in range(N)])  # note: `b_0` is always 0
    return a, b

### Symmetry properties of Fourier series <sup>[\[1\]](https://www.youtube.com/watch?v=YmYU-u_pFZY), [\[2\]](https://link.springer.com/content/pdf/bbm%3A978-1-4020-4818-0%2F1.pdf)</sup>

#### 1) Even-Function Symmetry
Examine the case, where the function is *even*, which means that

$$
    f(t) = f(-t).
$$

For any even periodic functions, the Fourier coefficients simplify to

$$
    a_{k}
    =
    \frac{4}{T} \int_{0}^{T/2} f(t) \cos \left( k \omega_{0} t \right) \operatorname{dt}
$$

$$
    b_{k}
    =
    0 \quad \text{for all}\ k
$$

#### 2) Odd-Function Symmetry
Examine the case, where the function is *odd*, which means that

$$
    f(t) = -f(t).
$$

In this case, the expressions for the Fourier coefficients are as follows:

$$
    a_{k}
    =
    0 \quad \text{for all}\ k
$$

$$
    b_{k}
    =
    \frac{4}{T} \int_{0}^{T/2} f(t) \sin \left( k \omega_{0} \right) \operatorname{dt}.
$$

#### 3) Time shifting/Translation
For every real $\tau$, if

$$
    f(t') = f(t \pm \tau),
$$

then

$$
    c_{k}
    =
    e^{\pm i k \omega_{0} \tau} \cdot \frac{1}{T} \int_{t_{0}}^{t_{0} + T} f(t) \cdot e^{-i k \omega_{0} t} \operatorname{dt}.
$$

#### 4) Time reversal
We examine the case, where

$$
    f \left( t' \right) = f(-t).
$$

Then

$$
    c_{k}
    =
    c_{-k}
    \quad\to\quad
    c_{k}
    =
    \frac{1}{T} \int_{t_{0}}^{t_{0} + T} f \left( t' \right) \cdot e^{-i (-k) \omega_{0} t'}  \operatorname{dt}'.
$$

#### 5) Frequency shifting/Frequency modulation
The frequency shifting property states that if

$$
    h(t)
    =
    e^{−i k_{0} \omega_{0} t} \cdot f(t),
$$

then

$$
    c_{k}
    =
    c_{k - k_{0}}
    \quad\to\quad
    c_{k}
    =
    \frac{1}{T} \int_{t_{0}}^{t_{0} + T} f(t) \cdot e^{-i (k-k_{0}) \omega_{0} t} \operatorname{dt}.
$$

#### 6) Differentiation
If

$$
    h(t)
    =
    \frac{df(t)}{dt}
$$

then

$$
    c_{k}
    =
    i k \omega_{0} \cdot \frac{1}{T} \int_{t_{0}}^{t_{0} + T} f(t) \cdot e^{-i k \omega_{0} t}\ \operatorname{dt}.
$$

#### 7) Multiplication/Amplitude modulation
If

$$
    h(t)
    =
    f(t) \cdot g(t)
$$

and the following is true:

$$
\begin{align}
    c_{k_{f}}
    &=
    \frac{1}{T} \int_{t_{0}}^{t_{0} + T} f(t) \cdot e^{-i k \omega_{0} t} \operatorname{dt}, \\
    c_{k_{g}}
    &=
    \frac{1}{T} \int_{t_{0}}^{t_{0} + T} g(t) \cdot e^{-i k \omega_{0} t} \operatorname{dt},
\end{align}
$$

then

$$
c_{k_{h}} = \sum_{i = -\infty}^{\infty} c_{i_{f}} c_{(k-i)_{g}}
$$